<a href="https://colab.research.google.com/github/khj951213/Machine-Learning/blob/main/ID3_13494315.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Machine Learning 31005
# Name: Hyun June Kim
# Student ID: 13494315
 

import pandas as pd
import numpy as np

_raw_data = """Outlook,Temperature,Humidity,Wind,Play
Sunny,Hot,High,Weak,No
Sunny,Hot,High,Strong,No
Overcast,Hot,High,Weak,Yes
Rain,Mild,High,Weak,Yes
Rain,Cool,Normal,Weak,Yes
Overcast,Cool,Normal,Strong,Yes
Sunny,Cool,Normal,Strong,Yes
Sunny,Mild,High,Weak,No
Rain,Cool,Normal,Weak,Yes
Sunny,Mild,Normal,Weak,Yes
Overcast,Mild,Normal,Strong,Yes
Overcast,Hot,Normal,Strong,Yes
"""

with open("sport_data.csv", "w") as f:
    f.write(_raw_data)
df = pd.read_csv("sport_data.csv")


In [ ]:
def compute_entropy(y):
    """
    :param y: The data samples of a discrete distribution
    """
    if len(y) < 2: #  a trivial case
        return 0
    freq = np.array( y.value_counts(normalize=True) )
    return -(freq * np.log2(freq + 1e-6)).sum() # the small eps for 
    # safe numerical computation 

In [ ]:
def compute_info_gain(samples, attr, target):
    values = samples[attr].value_counts(normalize=True)
    split_ent = 0
    for v, fr in values.iteritems():

        index = samples[attr]==v
        sub_ent = compute_entropy(target[index])
        split_ent += fr * sub_ent
    ent = compute_entropy(target)
    return ent - split_ent

class TreeNode:
    """
    A recursively defined data structure to store a tree.
    Each node can contain other nodes as its children
    """
    def __init__(self):
        self.children = {} # Sub nodes --
        # recursive, those elements of the same type (TreeNode)
        self.decision = None # Undecided
        self.split_feat_name = None # Splitting feature

    def pretty_print(self, prefix=''):
        if self.split_feat_name is not None:
            for k, v in self.children.items():
                v.pretty_print(f"{prefix}:When {self.split_feat_name} is {k}")
                #v.pretty_print(f"{prefix}:{k}:")
        else:
            print(f"{prefix}:{self.decision}")

    def predict(self, sample):
        if self.decision is not None:
            # uncomment to get log information of code execution
            print("Decision:", self.decision)
            return self.decision
        else: 
            # this node is an internal one, further queries about an attribute 
            # of the data is needed.
            attr_val = sample[self.split_feat_name]
            
            child = self.children[attr_val]
            # uncomment to get log information of code execution
            print("Testing ", self.split_feat_name, "->", attr_val)

            return child.predict(sample)
            # HINT: return prediction made by sub-tree

    def fit(self, X, y):
        """
        The function accepts a training dataset, from which it builds the tree 
        structure to make decisions or to make children nodes (tree branches) 
        to do further inquiries
        :param X: [n * p] n observed data samples of p attributes
        :param y: [n] target values
        """
        if len(X) == 0:
            # If the data is empty when this node is arrived, 
            # we just make an arbitrary decision
            self.decision = "Yes"
            return
        else: 
            unique_values = y.unique()
            if len(unique_values) == 1:
                self.decision= unique_values[0]
                return
            else:
                info_gain_max = 0

                for a in X.keys(): # Examine each attribute
                    aig = compute_info_gain(X, a, y)
                    print(f"attribute information gain for {a}: {aig}")
                    if aig > info_gain_max:
                        info_gain_max = aig
                        self.split_feat_name = a    
                    
                
                print(f"Split by {self.split_feat_name}, IG: {info_gain_max:.2f}")
                # 1st self.split_feat_name == humidity
                self.children = {}

                for v in X[self.split_feat_name].unique():
                    index = X[self.split_feat_name] == v # row that has v attribute
                    self.children[v] = TreeNode()
                    self.children[v].fit(X[index], y[index])
            

                    
# Test tree building
attrs = ['Outlook', 'Temperature', 'Humidity', 'Wind']
data = df[attrs]
target = df["Play"]

t = TreeNode()
t.fit(data, target)


attribute information gain for Outlook: 0.40671453519975437
attribute information gain for Temperature: 0.2075182687424375
attribute information gain for Humidity: 0.4067145351997544
attribute information gain for Wind: 0.006987753258770657
Split by Humidity, IG: 0.41
attribute information gain for Outlook: 0.9709485746841842
attribute information gain for Temperature: 0.019973094021878857
attribute information gain for Humidity: 0.0
attribute information gain for Wind: 0.17095001737734972
Split by Outlook, IG: 0.97


In [ ]:
# Test tree working
for i in [0,2,4]:
    print(f"Test predict sample[{i}]: \n{data.iloc[i]}\n\tTarget: {target.iloc[i]}")
    print(f"Making decision ...")
    pred = t.predict(data.iloc[i])


Test predict sample[0]: 
Outlook        Sunny
Temperature      Hot
Humidity        High
Wind            Weak
Name: 0, dtype: object
	Target: No
Making decision ...
Testing  Humidity -> High
Testing  Outlook -> Sunny
Decision: No
Test predict sample[2]: 
Outlook        Overcast
Temperature         Hot
Humidity           High
Wind               Weak
Name: 2, dtype: object
	Target: Yes
Making decision ...
Testing  Humidity -> High
Testing  Outlook -> Overcast
Decision: Yes
Test predict sample[4]: 
Outlook          Rain
Temperature      Cool
Humidity       Normal
Wind             Weak
Name: 4, dtype: object
	Target: Yes
Making decision ...
Testing  Humidity -> Normal
Decision: Yes


In [ ]:
t.pretty_print()

:When Humidity is High:When Outlook is Sunny:No
:When Humidity is High:When Outlook is Overcast:Yes
:When Humidity is High:When Outlook is Rain:Yes
:When Humidity is Normal:Yes
